In [2]:
import os
import re
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
import streamlit as st
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
load_dotenv()

True

In [3]:
huggingface_key = os.getenv("HUGGINGFACE_APIKEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [4]:
# doc = PyPDFLoader("/home/skynet/Documents/Gen_ai/GeetaGpt/data/bhagvatgeeta.pdf").load()

In [5]:

# def clean_text(text):
#     """Cleans the text by removing common escape sequences and unnecessary characters."""
#     # Remove common escape sequences (adjust as needed)
#     cleaned_text = re.sub(
#         r"[\x00-\x1F\x7F]", " ", text
#     )  # Removes control characters and replaces them with spaces
#     cleaned_text = re.sub(
#         r"\u200b", " ", cleaned_text
#     )  # Removes zero-width space and replaces it with a space

#     # Remove excessive whitespace
#     cleaned_text = re.sub(r"\s+", " ", cleaned_text)

#     return cleaned_text

In [6]:
# for document in doc:
#     document.page_content = clean_text(document.page_content)

In [7]:
# # remove empty pages
# doc = [document for document in doc if document.page_content]

In [8]:
# corpus = RecursiveCharacterTextSplitter(
#     chunk_size=1024, chunk_overlap=200
# ).split_documents(
#     doc
# )

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/skynet/miniforge3/envs/geetagpt_weaviate/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/skynet/miniforge3/envs/geetagpt_weaviate/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# db = FAISS.from_documents(corpus, embeddings)

In [11]:
# db.save_local("bhagvatgeeta_new")

In [12]:
db = FAISS.load_local("bhagvatgeeta_new",embeddings=embeddings,allow_dangerous_deserialization=True)

In [14]:
retriver = db.as_retriever()

In [15]:
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f129a7a9900>)

In [16]:
prompt_template = """

    Task: Answer questions based on the Bhagavad Gita teachings, incorporating the provided context.

    Instructions:

        Retrieve relevant information: Search the Bhagavad Gita for passages that are most relevant to the query, considering both the query itself and the provided context.
        Summarize key points: Extract the main ideas and concepts from the retrieved passages, taking into account the context for a more nuanced understanding.
        Generate a comprehensive response: Combine the summarized information with the query and context to create a coherent and informative answer.
        Mention references: Clearly indicate the chapter and verse numbers from which the information is extracted.
        Mention Sholkas: Clearly include relevant shlokas from the Gita to support your response.
        Provide accurate information: Ensure that the information provided is accurate and consistent with the teachings of the Gita, considering the context.
        Translate: Provide translations of the shlokas in Hindi to help explain their meaning.
        Stay focused: Keep your response focused on the query, considering both the Gita's teachings and the provided context.
        Be concise: Provide a clear and concise response that addresses the query effectively.

    Guidelines:

        Stay true to the text: Ensure that your responses are consistent with the teachings of the Bhagavad Gita.
        Provide context: If applicable, provide additional context from the Gita to support your answer, considering the provided context.
        Be respectful: Maintain a respectful and reverent tone when answering questions about the Bhagavad Gita.
        Be concise: Provide clear and concise responses that address the query directly.
        Stay focused: Ensure that your response is directly related to the Bhagavad Gita and avoids unrelated and out-of-context topics.
        Answering Style: Start with "Jai Shri Krishna" and end with "Jai Shri Krishna" in your answer.
        Out of scope: Avoid providing personal opinions or interpretations that are not supported by the text of the Gita.
        Out of Context: Avoid providing information that is not directly related to the query or the teachings of the Gita, considering the provided context.
        Inaccurate Information: Avoid providing information that is inaccurate or inconsistent with the teachings of the Gita.
        Not Sure: If you don't know the answer: Say "Jai Shri Krishna! I'm not sure, would you like me to look it up?"

    Context:
    {context}
    """

In [17]:
new_prompt = ChatPromptTemplate([prompt_template])

In [18]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
model = ChatGroq(model="gemma2-9b-it", max_tokens=4050,callbacks=[StreamingStdOutCallbackHandler()])

In [19]:
chain = create_stuff_documents_chain(model,new_prompt)

In [20]:
retrieval_chain = create_retrieval_chain(retriver, chain)

In [25]:
retrieval_chain.invoke({'input':'What is the Bhagavad Gita?'})['answer']

'Jai Shri Krishna! \n\nThe provided text highlights the importance of understanding the Bhagavad Gita in its purest form, as taught by Lord Krishna to Arjuna. \n\nThe text states: "If someone is fortunate enough to understand Bhagavad-gītā in that line of disciplic succession, without motivated interpretation, then he surpasses all studies of Vedic wisdom, and all scriptures of the world." (Bhagavad Gita As It Is, Purport)\n\nThis emphasizes the need for a genuine and unadulterated understanding of the Gita, passed down through a lineage of teachers (disciplic succession), free from personal biases or interpretations. \n\nThe text further states that the Bhagavad Gita is the essence of Vedic wisdom and contains truths not found elsewhere.  \n\nIt is considered the perfect theistic science because it is directly spoken by Lord Krishna himself.\n\nJai Shri Krishna! \n'

'/home/skynet/Documents/Gen_ai/GeetaGpt/notebooks/bhagvatgeeta_new'